In [2]:
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten ,Dropout
from keras.models import Sequential, Model
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import TensorBoard

from sklearn.metrics import classification_report, confusion_matrix

In [3]:
img_width = 224
img_height = 224
split = 0.2
train_samples = 5000 *(1-split)
validation_samples =  5000*split
epochs = 50
batch_size = 16
num_classes = 10

dataset = './dataset/training_set'
datatest = './dataset/test_set'

In [4]:
#Image Augementation train
image_generator = ImageDataGenerator(
    shear_range=0.05,
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    validation_split=split,
    )

train_generator = image_generator.flow_from_directory(
    dataset,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset="training",
    class_mode='categorical')

validation_generator = image_generator.flow_from_directory(
    dataset,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset="validation")

Found 360 images belonging to 10 classes.
Found 84 images belonging to 10 classes.


In [5]:
base_model = applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [6]:
top_model = Flatten(name="flatten")(base_model.output)
top_model = Dense(512,activation="relu")(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(num_classes,activation="softmax")(top_model)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
for layer in base_model.layers[:4]:
    layer.trainable = False   
    
final_model = Model(inputs=base_model.input,outputs=top_model)    

### VGG16 training :80% validation 20%

In [8]:
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
final_model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy","binary_crossentropy"])


#Model Fitting
callbacks = TensorBoard(log_dir='.\\Graph\\VGG16')
H = final_model.fit_generator(
	train_generator,
	steps_per_epoch=train_samples // batch_size,
	validation_data=validation_generator,
	validation_steps=validation_samples // batch_size,
	epochs=epochs,
    callbacks=[callbacks])

print("[INFO] serializing network...")
final_model.save("VGG16.h5")

[INFO] compiling model...


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Epoch 1/50
250/250 [==============================] - 98s 392ms/step - loss: 0.3129 - acc: 0.9005 - binary_crossentropy: 0.3129 - val_loss: 0.2564 - val_acc: 0.9049 - val_binary_crossentropy: 0.2564

Epoch 2/50
250/250 [==============================] - 92s 368ms/step - loss: 0.2351 - acc: 0.9165 - binary_crossentropy: 0.2351 - val_loss: 0.1753 - val_acc: 0.9384 - val_binary_crossentropy: 0.1753
Epoch 3/50
250/250 [==============================] - 91s 364ms/step - loss: 0.1697 - acc: 0.9374 - binary_crossentropy: 0.1697 - val_loss: 0.1349 - val_acc: 0.9545 - val_binary_crossentropy: 0.1349
Epoch 4/50
250/250 [==============================] - 91s 364ms/step - loss: 0.1329 - acc: 0.9498 - binary_crossentropy: 0.1329 - val_loss: 0.1165 - val_acc: 0.9578 - val_binary_crossentropy: 0.1165
Epoch 5/50
250/250 [==============================] - 91s 365ms/step - loss: 

Epoch 34/50
250/250 [==============================] - 91s 365ms/step - loss: 0.0067 - acc: 0.9973 - binary_crossentropy: 0.0067 - val_loss: 0.1682 - val_acc: 0.9643 - val_binary_crossentropy: 0.1682
Epoch 35/50
250/250 [==============================] - 91s 362ms/step - loss: 0.0064 - acc: 0.9977 - binary_crossentropy: 0.0064 - val_loss: 0.1395 - val_acc: 0.9732 - val_binary_crossentropy: 0.1395
Epoch 36/50
250/250 [==============================] - 91s 363ms/step - loss: 0.0069 - acc: 0.9970 - binary_crossentropy: 0.0069 - val_loss: 0.1660 - val_acc: 0.9667 - val_binary_crossentropy: 0.1660
Epoch 37/50
250/250 [==============================] - 92s 369ms/step - loss: 0.0068 - acc: 0.9973 - binary_crossentropy: 0.0068 - val_loss: 0.1571 - val_acc: 0.9673 - val_binary_crossentropy: 0.1571
Epoch 38/50
250/250 [==============================] - 91s 365ms/step - loss: 0.0065 - acc: 0.9971 - binary_crossentropy: 0.0065 - val_loss: 0.1693 - val_acc: 0.9671 - val_binary_crossentropy: 0.1693


In [9]:
image_test_generator = ImageDataGenerator(rescale=1. / 255)

testing_generator = image_test_generator.flow_from_directory(
    datatest,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
    )

Found 134 images belonging to 10 classes.


In [10]:
final_model.evaluate_generator(testing_generator,steps= 134 // batch_size+1,workers=12)


[0.18898979452118944, 0.9656716460612282, 0.18898979452118944]

In [11]:
testing_generator.reset()
Y_pred = final_model.predict_generator(testing_generator, 134 // batch_size+1,workers=12)

In [12]:
y_pred = np.argmax(Y_pred, axis=-1)
print('Confusion Matrix')
print(confusion_matrix(testing_generator.classes, y_pred))

Confusion Matrix
[[14  0  0  0  0  1  0  0  0  0]
 [ 0 15  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  2  0  0  0  0]
 [ 0  1  0 14  1  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  1  1  0]
 [ 2  0  0  1  0 13  1  1  0  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  3  0  0  0  0  0  9  0  0]
 [ 1  0  0  0  0  0  1  3 11  0]
 [ 1  0  0  0  0  0  0  0  2  9]]


In [13]:
batik_index = ['Batik Kawung','Batik Megamendung','Batik Nitik','Batik Parang','Batik Sido Luhur','Batik Truntum','Batik Udan Liris','Batik Gedok','Batik Ceplok','Batik Tambal']
print('Classification Report')
print(classification_report(testing_generator.classes, y_pred, target_names=batik_index))

Classification Report
                   precision    recall  f1-score   support

     Batik Kawung       0.78      0.93      0.85        15
Batik Megamendung       0.79      1.00      0.88        15
      Batik Nitik       1.00      0.50      0.67         4
     Batik Parang       0.93      0.88      0.90        16
 Batik Sido Luhur       0.91      0.83      0.87        12
    Batik Truntum       0.81      0.72      0.76        18
 Batik Udan Liris       0.88      1.00      0.93        14
      Batik Gedok       0.64      0.75      0.69        12
     Batik Ceplok       0.79      0.69      0.73        16
     Batik Tambal       1.00      0.75      0.86        12

         accuracy                           0.83       134
        macro avg       0.85      0.81      0.82       134
     weighted avg       0.84      0.83      0.83       134



In [165]:
from keras.models import load_model

In [166]:
loaded_model = load_model("MobileNetV2.h5")

In [168]:
testing_generator.reset()
loaded_model.evaluate_generator(testing_generator,steps= 134 // batch_size+1,workers=12)

[0.1276068142545757, 0.9529850625280124, 0.1276068142545757]

In [169]:
testing_generator.reset()
Y_pred = final_model.predict_generator(testing_generator, 134 // batch_size+1,workers=12)


In [170]:
y_pred = np.argmax(Y_pred, axis=-1)
print('Confusion Matrix')
print(confusion_matrix(testing_generator.classes, y_pred))

Confusion Matrix
[[11  0  0  0  1  2  0  0  1  0]
 [ 0 14  0  0  0  0  0  0  1  0]
 [ 0  0  2  0  0  2  0  0  0  0]
 [ 1  0  0 10  0  3  2  0  0  0]
 [ 0  1  0  0  8  1  1  0  1  0]
 [ 0  1  0  1  0 14  1  0  1  0]
 [ 0  1  0  0  0  0 13  0  0  0]
 [ 0  0  0  0  0  2  1  8  1  0]
 [ 1  0  1  0  1  2  1  1  9  0]
 [ 0  0  0  0  0  1  0  0  1 10]]


In [172]:
batik_index = ['Batik Kawung','Batik Megamendung','Batik Nitik','Batik Parang','Batik Sido Luhur','Batik Truntum','Batik Udan Liris','Batik Gedok','Batik Ceplok','Batik Tambal']
print('Classification Report')
print(classification_report(testing_generator.classes, y_pred, target_names=batik_index))

Classification Report
                   precision    recall  f1-score   support

     Batik Kawung       0.85      0.73      0.79        15
Batik Megamendung       0.82      0.93      0.87        15
      Batik Nitik       0.67      0.50      0.57         4
     Batik Parang       0.91      0.62      0.74        16
 Batik Sido Luhur       0.80      0.67      0.73        12
    Batik Truntum       0.52      0.78      0.62        18
 Batik Udan Liris       0.68      0.93      0.79        14
      Batik Gedok       0.89      0.67      0.76        12
     Batik Ceplok       0.60      0.56      0.58        16
     Batik Tambal       1.00      0.83      0.91        12

         accuracy                           0.74       134
        macro avg       0.77      0.72      0.74       134
     weighted avg       0.77      0.74      0.74       134

